<a href="https://colab.research.google.com/github/DataWitchcraft/python4sci/blob/main/10_Pandas_Data_Manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data manipulation with Pandas library

In the following part of the course, we are going to work with tabular data - i.e. data that you probably know from your favourite (or un-favourite) spreadsheet. Usually each row of such a table corresponds to some thing, an instance of something, or some observation. The individual columns then contain the properties or measured quantities characteristic of those things.

In the Python world, the pandas library is most commonly used to process tabular data. It allows you to read data from many formats (including Excel workbooks), modify them in various ways, count columns very efficiently, directly examine some statistical indicators, and visualize the results. 

For more information about the pandas library, visit https://pandas.pydata.org/

In [1]:
import pandas as pd

This imports the pandas library, but it will not be available under its usual name, but under the alias `pd`. Usually, we try to avoid aliases because they reduce the readability of the code for other programmers. For data analytics, it is different because using a single alias, which is also very common, saves us a lot of typing.

In [2]:
import requests
from pathlib import Path

datapath = Path('./data')
datapath.mkdir(exist_ok=True)

to_download = [
    'https://raw.githubusercontent.com/DataWitchcraft/python4sci/main/data/cze.csv',
    'https://raw.githubusercontent.com/DataWitchcraft/python4sci/main/data/countries.csv',
    'https://raw.githubusercontent.com/DataWitchcraft/python4sci/main/data/surveys.csv',
    'https://raw.githubusercontent.com/DataWitchcraft/python4sci/main/data/species.csv',
]
    
for link in to_download:
    name = link.rsplit("/")[-1]
    filepath = datapath / name
    if not filepath.exists():
        r = requests.get(link)
        filepath.write_bytes(r.content)
        print(f'File {name} downloaded.')
    else:
        print(f'File {name} already exists.')
        

File cze.csv already exists.
File countries.csv already exists.
File surveys.csv already exists.
File species.csv already exists.


## Reading the data

For reading data, Pandas has a number of `read_*` functions that can handle many different formats. A fairly common format is `CSV` ("comma-separated values" - wiki), in which each record corresponds to one line, and the individual record properties are separated by commas (or another character).

For this section, we will be using the Portal Teaching data, a subset of the data from Ernst et al Long-term monitoring and experimental manipulation of a [Chihuahuan Desert ecosystem near Portal, Arizona, USA](http://www.esapubs.org/archive/ecol/E090/118/default.htm).

The dataset stored in `surveys.csv` file contains data on animals caught in sites in the studied area. 


|Column	|Description|
|:-------|:------------------|
|record_id	|Unique id for the observation|
|month	|month of observation|
|day	|day of observation|
|year	|year of observation|
|plot_id	|ID of a particular site|
|species_id	|2-letter code|
|sex	|sex of animal (“M”, “F”)|
|hindfoot_length	|length of the hindfoot in mm|
|weight	|weight of the animal in grams|

In [3]:
surveys_df = pd.read_csv("data/surveys.csv")

In [4]:
surveys_df

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN
...,...,...,...,...,...,...,...,...,...
35544,35545,12,31,2002,15,AH,NaN,NaN,NaN
35545,35546,12,31,2002,15,AH,NaN,NaN,NaN
35546,35547,12,31,2002,10,RM,F,15.0,14.0
35547,35548,12,31,2002,7,DO,M,36.0,51.0


If everything worked as it should, you should see a relatively nicely formatted table. The basic jupyter notebook view will show you the first five and last five rows (who would risk having thousands of rows cluttering up their browser window?) in the table, along with information about the total number of rows and columns. In this case, the table contains a total of 9 properties (named columns) for 35549 different records (numbered rows).

⚠️ Warning. Usually there are problems with the input data - for example, they don't have columns described (or have them described in a weird way), they use strange data separators or decimal decimal, many rows have missing values (or are misspelled), ...

## Basic Pandas object types - DataFrame, Series, Index

In [5]:
type(surveys_df)

pandas.core.frame.DataFrame

`DataFrame` is very similar to a spreadsheet in its functions, but it is important to realize where this parallel ends. Unlike Excel or LibreOffice workbooks, DataFrame contains "only" data and does not store any formatting.

`DataFrame` behaves similarly to a `dict` - if you put a key in square brackets, you get a named column. In fact, square brackets allow you to select from tables based on various other criteria, but we'll get to that later.

In [6]:
hindfoot = surveys_df["hindfoot_length"]
hindfoot

0        32.0
1        33.0
2        37.0
3        36.0
4        35.0
         ... 
35544     NaN
35545     NaN
35546    15.0
35547    36.0
35548     NaN
Name: hindfoot_length, Length: 35549, dtype: float64

### Series

In [7]:
type(hindfoot)

pandas.core.series.Series

The columns are of the `Series` type. This type looks like a `list`. Let's check if it behaves like that:

In [8]:
hindfoot[0]

32.0

In [9]:
hindfoot[-4:]

35545     NaN
35546    15.0
35547    36.0
35548     NaN
Name: hindfoot_length, dtype: float64

**Your turn:**
Try to apply some other operations with lists that you already know on the hindfoot Series. Sometimes it works, sometimes it doesn't.

You can also convert the data between lists and Series. The easiest way to create your own Series (outside the context of a table!) is to create an instance of this class with a list as an argument:

In [10]:
numbers = pd.Series([1, 2, 3])
numbers

0    1
1    2
2    3
dtype: int64

or the other way round:

In [11]:
numbers.tolist()  

[1, 2, 3]

So, what is the difference between Series and list? 

In [12]:
hindfoot

0        32.0
1        33.0
2        37.0
3        36.0
4        35.0
         ... 
35544     NaN
35545     NaN
35546    15.0
35547    36.0
35548     NaN
Name: hindfoot_length, Length: 35549, dtype: float64

Each column has **values**, **index**, **name**, **size** and **dtype**

#### Values

In [13]:
hindfoot.values[:5]

array([32., 33., 37., 36., 35.])

In [14]:
type(hindfoot.values)

numpy.ndarray

#### Type of values

In [15]:
hindfoot.dtype

dtype('float64')

Unlike in lists, all Series elements should be of the same type (if not, the nearest common type is chosen).

#### Index

In [16]:
hindfoot.index

RangeIndex(start=0, stop=35549, step=1)

You can access the elements of the list by numerical order (0 - first element, 1 - second, ...), you select from the dictionary according to the key, pandas introduces a generalized index, which can be numeric, string or even based on date/time. 

#### Name

In [17]:
hindfoot.name

'hindfoot_length'

Series may or may not have a name. Note that the name is not related to the name of the variable in which you store the Series.

#### Size

In [18]:
hindfoot.size

35549

#### Your turn:

Find the values of the `.name`, `.index`, `.dtype`, `.values` and `.size` attributes of the `numbers` object. Do you notice anything interesting? 

You can specify some of these attributes when creating Series objects:

In [19]:
age = pd.Series(
    [27, 65, 14],
    name="Age",
    index=["Alice", "Bob", "Charlie"],
    dtype=float,
)
age

Alice      27.0
Bob        65.0
Charlie    14.0
Name: Age, dtype: float64

#### Index

In [20]:
hindfoot.index  # numerical index, sorted from 0

RangeIndex(start=0, stop=35549, step=1)

In [21]:
age.index 

Index(['Alice', 'Bob', 'Charlie'], dtype='object')

In [22]:
events = pd.Series(
    ["Some event", "Some other event", "Event 3"],
    index = pd.Index([2022, 1800, 1967], name="year")   # Index can have a name
)
events

year
2022          Some event
1800    Some other event
1967             Event 3
dtype: object

In [23]:
events.index  # values may not be sorted

Int64Index([2022, 1800, 1967], dtype='int64', name='year')

In [24]:
events_prec = pd.Series(
    ["Some event", "Some other event", "Event 3"],
    index = pd.DatetimeIndex(["2022-10-20", "1800-11-11", "1967-12-31"])   
)
events_prec.index

DatetimeIndex(['2022-10-20', '1800-11-11', '1967-12-31'], dtype='datetime64[ns]', freq=None)

The index values can be used to access elements of the Series:

In [25]:
age["Alice"]

27.0

**Your turn:**
What is the index of surveys_df?

### DataFrame

In [26]:
surveys_df

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN
...,...,...,...,...,...,...,...,...,...
35544,35545,12,31,2002,15,AH,NaN,NaN,NaN
35545,35546,12,31,2002,15,AH,NaN,NaN,NaN
35546,35547,12,31,2002,10,RM,F,15.0,14.0
35547,35548,12,31,2002,7,DO,M,36.0,51.0


`DataFrame` is two-dimensional and contains two indexes in addition to values (`.values`) - one for rows and one for columns:

In [27]:
surveys_df.values

array([[1, 7, 16, ..., 'M', 32.0, nan],
       [2, 7, 16, ..., 'M', 33.0, nan],
       [3, 7, 16, ..., 'F', 37.0, nan],
       ...,
       [35547, 12, 31, ..., 'F', 15.0, 14.0],
       [35548, 12, 31, ..., 'M', 36.0, 51.0],
       [35549, 12, 31, ..., nan, nan, nan]], dtype=object)

In [28]:
surveys_df.columns

Index(['record_id', 'month', 'day', 'year', 'plot_id', 'species_id', 'sex',
       'hindfoot_length', 'weight'],
      dtype='object')

In [29]:
surveys_df.index

RangeIndex(start=0, stop=35549, step=1)

In [30]:
surveys_df.shape

(35549, 9)

There are several ways to construct a DataFrame:

In [31]:
pd.DataFrame({
    "number": [1, 2, 3],
    "letter": ["a", "b", "c"]
})

,number,letter
0,1,a
1,2,b
2,3,c


In [32]:
items = pd.DataFrame([
        {"name": "rubber duck", "price": 12.90, "origin": "US"},
        {"name": "pencil", "price": 2.90, "origin": "SK"},
        {"name": "teddy bear", "price": 22.90, "origin": "CZ"},   
    ],
    index=["item1", "item2", "item3"]
)
items

,name,price,origin
item1,rubber duck,12.9,US
item2,pencil,2.9,SK
item3,teddy bear,22.9,CZ


## Accessing data

In [33]:
items = items.set_index("name").sort_index()
items

,price,origin
name,,
pencil,2.9,SK
rubber duck,12.9,US
teddy bear,22.9,CZ


- `set_index()` returns a table in which one of the columns is used as an index

- `sort_index()` returns a table that contains the same index but sorted

In [34]:
surveys_df[4:7]

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
4,5,7,16,1977,3,DM,M,35.0,NaN
5,6,7,16,1977,1,PF,M,14.0,NaN
6,7,7,16,1977,2,PE,F,NaN,NaN


In [35]:
surveys_df[["species_id", "sex"]]

,species_id,sex
0,NL,M
1,NL,M
2,DM,F
3,DM,M
4,DM,M
...,...,...
35544,AH,NaN
35545,AH,NaN
35546,RM,F
35547,DO,M


We can select specific ranges of our data in both the row and column directions using either label or integer-based indexing.

- `loc` is primarily label based indexing. Integers may be used but they are interpreted as a label.
- `iloc` is primarily integer based indexing

To select a subset of rows and columns from our DataFrame, we can use the iloc method. For example, we can select month, day and year (columns 2, 3 and 4 if we start counting at 1), like this:

In [36]:
# iloc[row slicing, column slicing]
surveys_df.iloc[0:3, 1:4]

,month,day,year
0,7,16,1977
1,7,16,1977
2,7,16,1977


In [37]:
items.loc[["pencil", "teddy bear"],]

,price,origin
name,,
pencil,2.9,SK
teddy bear,22.9,CZ


In [38]:
surveys_df.loc[0, ["species_id", "plot_id", "weight"]]

species_id     NL
plot_id         2
weight        NaN
Name: 0, dtype: object

Warning: Labels must be found in the `DataFrame` or you will get a `KeyError`.

Indexing by labels `loc` differs from indexing by integers `iloc`. With `loc`, both the start bound and the stop bound are inclusive. When using `loc`, integers can be used, but the integers refer to the index label and not the position. For example, using `loc` and select 1:4 will get a different result than using `iloc` to select rows 1:4.



In [39]:
surveys_df.iloc[1:4]

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN


In [40]:
surveys_df.loc[1:4]

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN


**Your turn:**

What happens when you execute:

- `surveys_df[0:1]`

- `surveys_df[:4]`

- `surveys_df[:-1]`

What happens when you call:

- `surveys_df.iloc[0:4, 1:4]`

- `surveys_df.loc[0:4, 1:4]`


`head`, `tail`, and `sample` are three convenient functions that select the first, last, or random rows of a table (all three have an optional parameter specifying the number of rows required):

In [41]:
surveys_df.head()

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,1,7,16,1977,2,NL,M,32.0,NaN
1,2,7,16,1977,3,NL,M,33.0,NaN
2,3,7,16,1977,2,DM,F,37.0,NaN
3,4,7,16,1977,7,DM,M,36.0,NaN
4,5,7,16,1977,3,DM,M,35.0,NaN


In [42]:
surveys_df.tail(3)

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
35546,35547,12,31,2002,10,RM,F,15.0,14.0
35547,35548,12,31,2002,7,DO,M,36.0,51.0
35548,35549,12,31,2002,5,NaN,NaN,NaN,NaN


In [43]:
surveys_df.sample()

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
4667,4668,7,8,1981,6,DS,M,49.0,95.0


### Subsetting data using criteria:

In [44]:
surveys_df[(surveys_df.year >= 1982) & (surveys_df.year <= 1984)]

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
5157,5158,1,1,1982,1,DO,F,37.0,49.0
5158,5159,1,1,1982,5,DM,F,34.0,42.0
5159,5160,1,1,1982,3,RM,F,16.0,11.0
5160,5161,1,1,1982,5,NL,F,33.0,193.0
5161,5162,1,1,1982,3,RM,F,17.0,10.0
...,...,...,...,...,...,...,...,...,...
9784,9785,12,31,1984,2,DO,M,33.0,45.0
9785,9786,12,31,1984,5,DM,F,35.0,33.0
9786,9787,12,31,1984,1,DM,M,36.0,51.0
9787,9788,12,31,1984,11,DM,M,36.0,35.0


In [45]:
surveys_df.query("weight > 250 & year == 1987")

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
12298,12299,2,1,1987,2,NL,M,32.0,253.0
12457,12458,3,2,1987,2,NL,M,33.0,259.0
12601,12602,4,6,1987,2,NL,M,34.0,260.0
12728,12729,4,26,1987,2,NL,M,32.0,270.0
12870,12871,5,28,1987,2,NL,M,32.0,278.0
13024,13025,7,1,1987,2,NL,M,33.0,260.0
13113,13114,7,26,1987,2,NL,M,NaN,269.0


### Sorting

In [46]:
surveys_df.sort_values("weight")

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
9908,9909,1,20,1985,15,RM,F,15.0,4.0
4289,4290,4,6,1981,4,PF,NaN,NaN,4.0
9793,9794,1,19,1985,24,RM,M,16.0,4.0
9789,9790,1,19,1985,16,RM,F,16.0,4.0
5345,5346,2,22,1982,21,PF,F,14.0,4.0
...,...,...,...,...,...,...,...,...,...
35530,35531,12,31,2002,13,PB,F,27.0,NaN
35543,35544,12,31,2002,15,US,NaN,NaN,NaN
35544,35545,12,31,2002,15,AH,NaN,NaN,NaN
35545,35546,12,31,2002,15,AH,NaN,NaN,NaN


In [47]:
surveys_df.sort_values("hindfoot_length", ascending=False)

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
10573,10574,7,23,1985,12,NL,NaN,70.0,NaN
30424,30425,3,4,2000,1,DO,F,64.0,35.0
1693,1694,3,31,1979,8,DS,F,58.0,123.0
22048,22049,2,4,1995,2,DO,M,58.0,51.0
4448,4449,5,4,1981,2,DS,F,57.0,136.0
...,...,...,...,...,...,...,...,...,...
35527,35528,12,31,2002,13,US,NaN,NaN,NaN
35543,35544,12,31,2002,15,US,NaN,NaN,NaN
35544,35545,12,31,2002,15,AH,NaN,NaN,NaN
35545,35546,12,31,2002,15,AH,NaN,NaN,NaN


**Your turn:**
Find 10 heaviest animals cought in 1985.

## DataFrame manipulation
In this example, we are going to use the items `DataFrame` we have created before:

In [48]:
items

,price,origin
name,,
pencil,2.9,SK
rubber duck,12.9,US
teddy bear,22.9,CZ


### Adding a new column
When we want to add a new column (Series), we assign it to the DataFrame as a dictionary value - that is, in square brackets with the column name. The good news is that, just like in the constructor, pandas can handle both Series and a regular list.

Let's add color:

In [49]:
items["color"] = ["red", "blue", "blue"]
items

,price,origin,color
name,,,
pencil,2.9,SK,red
rubber duck,12.9,US,blue
teddy bear,22.9,CZ,blue


💡 In this case we directly modified an existing `DataFrame`. By default, most methods / operations in pandas (you already know e.g. `set_index`) always return a new object with the applied modification, leaving the original object unchanged. This is a good habit that we will follow. Column assignment is one of the exceptions to this otherwise accepted rule.

However, DataFrame still provides an `assign` method that does not modify the table, but creates a copy of it with the added (or replaced) columns. If you want to avoid the annoyance of keeping track of which table you've changed or not, `assign` is highly recommended.

By the way, you can create a copy of a table at any time using the `copy` method - this is useful when writing functions where the input table is modified for various reasons.

In [50]:
items.assign(is_new=[True, False, False],
             size=["S", "S", "M"]
            )
# the object items is unchanged

,price,origin,color,is_new,size
name,,,,,
pencil,2.9,SK,red,True,S
rubber duck,12.9,US,blue,False,S
teddy bear,22.9,CZ,blue,False,M


In [51]:
items2 = items.copy()
items2["you_can_eat_it"] = [False, False, False]
items2
# the object items is still unchanged

,price,origin,color,you_can_eat_it
name,,,,
pencil,2.9,SK,red,False
rubber duck,12.9,US,blue,False
teddy bear,22.9,CZ,blue,False


####  Adding a new row


In [52]:
items.loc["wooden train"] = ["25", "GE", "green"]   # List of values in row
items

,price,origin,color
name,,,
pencil,2.9,SK,red
rubber duck,12.9,US,blue
teddy bear,22.9,CZ,blue
wooden train,25,GE,green


### Cell value change
The `.loc` and `.iloc` "indexers" with two arguments in square brackets refer directly to a specific cell, and assigning them (again, like in the dictionary) will write the value to the appropriate location. You just need to keep the order (row, column).

Let's change the price of rubber duck:

In [53]:
items.loc["rubber duck", "price"] = 14.2
items

,price,origin,color
name,,,
pencil,2.9,SK,red
rubber duck,14.2,US,blue
teddy bear,22.9,CZ,blue
wooden train,25,GE,green


Warning: Similarly to the dictionary, but perhaps somewhat counter-intuitively, it is possible to write a value into a row and column that do not exist!

In [54]:
items_bad = items.copy()
items_bad.loc["teddy bear", "colour"] = "black"
items_bad

,price,origin,color,colour
name,,,,
pencil,2.9,SK,red,NaN
rubber duck,14.2,US,blue,NaN
teddy bear,22.9,CZ,blue,black
wooden train,25,GE,green,NaN


### Deleting a row
The `drop` method is used to remove a column or row from the `DataFrame`. Its first argument expects the index of one or more rows or columns that you want to remove. The axis argument indicates in which dimension the operation is to be applied. You can use either the number 0 or 1 (corresponding to the order from zero in which keys are given when referencing cells), or the name of the dimension:

Axis:

- 0 or "index" → rows
- 1 or "columns" → columns
This argument is also used by many other methods and functions, so make sure you understand it

Let's delete pencil (for the drop method, the default value of the axis argument is 0, so we don't need to write it):

In [55]:
items = items.drop("pencil")
items

,price,origin,color
name,,,
rubber duck,14.2,US,blue
teddy bear,22.9,CZ,blue
wooden train,25,GE,green


### Deleting a column
For a column, the drop method works very similarly, only this time the axis argument must be specified.

Let's remove color:

In [56]:
items = items.drop("color", axis="columns")  
items

,price,origin
name,,
rubber duck,14.2,US
teddy bear,22.9,CZ
wooden train,25,GE


The `drop` method, in accordance with the above convention, returns a new `DataFrame` (and therefore the result of the operation must be assigned to `items`). If you want to operate directly on the table, you can use the `del` command (it works the same as the dictionary) or ask the panda gods for forgiveness and add the `inplace=True` argument (this argument can, unfortunately, be used for many other operations):

In [57]:
# at your own risk :)
#  1)
# del items["origin"]

#  2)
# items.drop("origin", axis=1, inplace=True)

**Your turn:** 
- Add a new item
- Calculate price_with_tax as 1.2 * price and add it as a new column.

## Non defined values 

Where are the non defined values and what to do with them?

In [58]:
surveys_df.isna()

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...
35544,False,False,False,False,False,False,True,True,True
35545,False,False,False,False,False,False,True,True,True
35546,False,False,False,False,False,False,False,False,False
35547,False,False,False,False,False,False,False,False,False


Using `sum()` for `True` and `False` is a useful trick, `True` counts as 1, `False` as 0:

In [59]:
surveys_df.isna().sum()

record_id             0
month                 0
day                   0
year                  0
plot_id               0
species_id          763
sex                2511
hindfoot_length    4111
weight             3266
dtype: int64

We can also select rows where there is at least one `NaN`. You will remember that you can select rows using a condition. However, there is a problem that we have more than one of these rows - one for each column. But we can use `.any(axis=1)` to select rows where there is at least one `NaN`:

In [60]:
surveys_df.loc[surveys_df.isna().any(axis=1)].sample(5)

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
6031,6032,6,28,1982,6,DM,M,NaN,49.0
27555,27556,3,1,1998,20,PM,F,21.0,NaN
19195,19196,10,11,1991,10,NaN,NaN,NaN,NaN
6213,6214,7,25,1982,6,PE,M,NaN,24.0
6225,6226,7,26,1982,12,DM,M,NaN,49.0


Number of non-missing values:

In [61]:
surveys_df.count()

record_id          35549
month              35549
day                35549
year               35549
plot_id            35549
species_id         34786
sex                33038
hindfoot_length    31438
weight             32283
dtype: int64

In general, we have three basic options what to do with missing values.

Do nothing, i.e. leave the missing data missing. This is, perhaps surprisingly, often a good choice, as many functions will handle missing data correctly. This is different from numpy, where functions typically don't like `NaN`.

Remove rows with missing records. This is what the `dropna` method is for.

Replace the missing data with some suitable value. What the appropriate values are depends on the nature of the data and what we want to do with the data. Sometimes it is appropriate to replace missing values with some "typical" value, such as an average. For time series, it is usually more logical to replace with a neighborhood value (previous or next). You can read about imputation at https://scikit-learn.org/stable/modules/impute.html and maybe learn more later. The `fillna` method is used to replace values by a defined value or by neighboring values.

In [62]:
example_with_nan = pd.DataFrame({"A": [1, 2, pd.NA, 4, 5], "B": [1, 2, 3, pd.NA, 5]})
example_with_nan

,A,B
0,1,1
1,2,2
2,<NA>,3
3,4,<NA>
4,5,5


In [63]:
example_with_nan.dropna()

,A,B
0,1,1
1,2,2
4,5,5


## Basic stats

In [64]:
surveys_df.describe()

,record_id,month,day,year,plot_id,hindfoot_length,weight
count,35549.000000,35549.000000,35549.000000,35549.000000,35549.000000,31438.000000,32283.000000
mean,17775.000000,6.474022,16.105966,1990.475231,11.397001,29.287932,42.672428
std,10262.256696,3.396583,8.256691,7.493355,6.799406,9.564759,36.631259
min,1.000000,1.000000,1.000000,1977.000000,1.000000,2.000000,4.000000
25%,8888.000000,4.000000,9.000000,1984.000000,5.000000,21.000000,20.000000
50%,17775.000000,6.000000,16.000000,1990.000000,11.000000,32.000000,37.000000
75%,26662.000000,9.000000,23.000000,1997.000000,17.000000,36.000000,48.000000
max,35549.000000,12.000000,31.000000,2002.000000,24.000000,70.000000,280.000000


For each column we see several summary (statistical data).

- `count` indicates the number of values.
- `mean` is the mean value, calculated as an arithmetic average.
- `std` is the standard deviation, which shows the variance of the data - how much we can expect the data in the set to differ from the mean.
- `min` and `max` are the smallest and largest values in the column.
- Percentages indicate quantiles, `25%` and `75%` are the first and third "quartile" values. If we sort the column by size, a quarter of the data will be smaller than the first quartile value and a quarter of the data will be larger than the third quartile value.
- The `50%` is referred to as the median - half of the data is less than the median (and the other half, of course, is again greater than the median).


In [65]:
surveys_df[surveys_df["year"] == 1990].describe()  # we can use describe on a subset of data

,record_id,month,day,year,plot_id,hindfoot_length,weight
count,1311.000000,1311.000000,1311.000000,1311.0,1311.000000,1155.000000,1209.000000
mean,17534.000000,5.699466,20.770404,1990.0,10.594966,28.408658,35.483044
std,378.597412,3.783006,7.008059,0.0,6.447079,9.311903,27.764739
min,16879.000000,1.000000,6.000000,1990.0,1.000000,13.000000,6.000000
25%,17206.500000,2.000000,16.000000,1990.0,4.000000,18.000000,14.000000
50%,17534.000000,5.000000,22.000000,1990.0,10.000000,34.000000,37.000000
75%,17861.500000,9.000000,25.000000,1990.0,15.000000,37.000000,46.000000
max,18189.000000,12.000000,30.000000,1990.0,24.000000,55.000000,243.000000


## Groups in Pandas

One of the basic procedures in data analysis is to divide the data into groups, apply some operation to each group and finally combine the results into a suitable dataset. This procedure is referred to as split-apply-combine. Groups are often defined by some (categorical) variable, e.g., it could be color, gender, or continent. However, groups can also be created by inference, e.g., using range or time series properties. 

#### 1: Split
To divide the data into groups, `groupby` method is used in Pandas. The simplest and perhaps most common use is to group by an existing column in the table.

In [66]:
surveys_by_sex = surveys_df.groupby("sex")
surveys_by_sex

#### 2 + 3: Apply + Combine
We need to specify what to do with the created groups

In [67]:
surveys_by_sex.count()  # number of values

,record_id,month,day,year,plot_id,species_id,hindfoot_length,weight
sex,,,,,,,,
F,15690,15690,15690,15690,15690,15690,14894,15303
M,17348,17348,17348,17348,17348,17348,16476,16879


In [68]:
surveys_by_sex.size()  # size of each group

sex
F    15690
M    17348
dtype: int64

**Your turn:**
Divide the data by species_id and print number of records in each group.
Can you order the groups by their size?

In [69]:
surveys_by_species_and_sex_count = surveys_df.groupby(["species_id", "sex"]).count()
surveys_by_species_and_sex_count.head()

record_id  month   day  year  plot_id  hindfoot_length  weight
species_id sex                                                                
AH         M            1      1     1     1        1                0       0
BA         F           31     31    31    31       31               31      31
           M           14     14    14    14       14               14      14
DM         F         4554   4554  4554  4554     4554             4302    4440
           M         5969   5969  5969  5969     5969             5658    5808

In [70]:
surveys_by_species_and_sex_count.index[:5]

MultiIndex([('AH', 'M'),
            ('BA', 'F'),
            ('BA', 'M'),
            ('DM', 'F'),
            ('DM', 'M')],
           names=['species_id', 'sex'])

The created groups are defined by the pair of values `species_id` and `sex`, that's why `MultiIndex` is used. In this course, we are not going cover `MultiIndex`, if you want to learn more about MultiIndex, check the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

To remove `MultiIndex`, we can use `.reset_index()` or use `groupby` with `as_index=False`

In [71]:
surveys_by_species_and_sex_count.reset_index().head()

,species_id,sex,record_id,month,day,year,plot_id,hindfoot_length,weight
0,AH,M,1,1,1,1,1,0,0
1,BA,F,31,31,31,31,31,31,31
2,BA,M,14,14,14,14,14,14,14
3,DM,F,4554,4554,4554,4554,4554,4302,4440
4,DM,M,5969,5969,5969,5969,5969,5658,5808


In [72]:
surveys_df.groupby(["species_id", "sex"], as_index=False).count().head()

,species_id,sex,record_id,month,day,year,plot_id,hindfoot_length,weight
0,AH,M,1,1,1,1,1,0,0
1,BA,F,31,31,31,31,31,31,31
2,BA,M,14,14,14,14,14,14,14
3,DM,F,4554,4554,4554,4554,4554,4302,4440
4,DM,M,5969,5969,5969,5969,5969,5658,5808


#### Another use of DataFrameGroupBy object:

In [73]:
groups = surveys_df[surveys_df["species_id"].isin(['DM', 'PP', 'DO'])] \
              .groupby("species_id")  # select just a few species
next(iter(groups))

('DM',
        record_id  month  day  year  plot_id species_id sex  hindfoot_length  \
 2              3      7   16  1977        2         DM   F             37.0   
 3              4      7   16  1977        7         DM   M             36.0   
 4              5      7   16  1977        3         DM   M             35.0   
 7              8      7   16  1977        1         DM   M             37.0   
 8              9      7   16  1977        1         DM   F             34.0   
 ...          ...    ...  ...   ...      ...        ...  ..              ...   
 35532      35533     12   31  2002       14         DM   F             36.0   
 35533      35534     12   31  2002       14         DM   M             37.0   
 35534      35535     12   31  2002       14         DM   M             37.0   
 35535      35536     12   31  2002       14         DM   F             35.0   
 35536      35537     12   31  2002       14         DM   F             36.0   
 
        weight  
 2         NaN

In [74]:
for species_id, df in groups:
    df.to_excel(species_id + ".xlsx")

**Bonus:** read all the xlsx we created and put them together into one dataframe

(for `pathlib.glob` see https://docs.python.org/3/library/pathlib.html#pathlib.Path.glob)

In [75]:
p = Path('.')
filenames = p.glob('*.xlsx')
new_df = pd.DataFrame()
for filename in filenames:
    tmp_df = pd.read_excel(filename)
    new_df = new_df.append(tmp_df)

In [76]:
new_df

,Unnamed: 0,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
0,17,18,7,16,1977,2,PP,M,22.0,NaN
1,28,29,7,17,1977,11,PP,M,NaN,NaN
2,48,49,7,18,1977,12,PP,F,17.0,NaN
3,88,89,8,20,1977,12,PP,F,20.0,15.0
4,107,108,8,20,1977,11,PP,M,21.0,NaN
...,...,...,...,...,...,...,...,...,...,...
3022,35502,35503,12,31,2002,6,DO,M,33.0,32.0
3023,35515,35516,12,31,2002,11,DO,F,35.0,52.0
3024,35517,35518,12,31,2002,11,DO,M,36.0,38.0
3025,35526,35527,12,31,2002,13,DO,F,33.0,43.0


#### Other aggregation functions
So far we have only aggregated using `.size` or `.count`, but there are many other aggregation methods, especially numeric ones.

To make it easier to work with aggregations, we will use the `.agg` method, which accepts a dictionary. This dictionary specifies what we want to aggregate (the key) and how we want to aggregate it (the value). With that said, there can be multiple aggregation methods for a single column at once.

In [77]:
surveys_df.groupby("species_id").agg({
    "hindfoot_length": ["min", "max"],
    "weight": "mean",
    "plot_id": "nunique"}
).head()

hindfoot_length       weight plot_id
                       min   max   mean nunique
species_id                                     
AB                     NaN   NaN    NaN      24
AH                    31.0  35.0    NaN      24
AS                     NaN   NaN    NaN       2
BA                     6.0  16.0    8.6       8
CB                     NaN   NaN    NaN      18

## Pivot tables

If we aggregate over multiple columns, we may find that the DataFrame becomes very long, which may not be convenient to handle. If you've ever worked with such data in Excel, you may be familiar with the pivot tables.

Before we start pivoting, let's create a small dataset - it will be an aggregation based on the species and gender.

In [78]:
summary = surveys_df.groupby(["species_id", "sex"], as_index=False)["weight"].mean()
summary.head(10)

,species_id,sex,weight
0,AH,M,NaN
1,BA,F,9.161290
2,BA,M,7.357143
3,DM,F,41.609685
4,DM,M,44.353134
5,DO,F,48.531250
6,DO,M,49.135102
7,DS,F,117.749548
8,DS,M,122.290984
9,NL,F,154.282209


To create a pivot table, whe have to specify three things:

- which column we want in the index
- which column will be in the columns
- which column to assign to the values in the DataFrame

In [79]:
summary.pivot(index="species_id", columns="sex", values="weight")

sex,F,M
species_id,,
AH,NaN,NaN
BA,9.161290,7.357143
DM,41.609685,44.353134
DO,48.531250,49.135102
DS,117.749548,122.290984
NL,154.282209,165.652893
OL,31.065817,32.068410
OT,24.830904,23.695382
OX,21.000000,21.000000


The limitation of the `pivot` method is that it can only rotate the DataFrame, but if one of the created fields has multiple values, it will throw you an error because it doesn't know how to aggregate them.

For more complex aggregations there's a similarly named `pivot_table`.

`pivot_table` offers the possibility to assign multiple columns to one of the dimensions (index or columns) and then creates a multi-level index. It is also possible to specify which function is applied if there are multiple values per cell.

For mor details, see the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

In [80]:
summary = surveys_df[surveys_df["species_id"].isin(['DM', 'PP', 'DO'])] \
    .groupby(['species_id', 'sex', 'plot_id'], as_index=False).size()
summary

,species_id,sex,plot_id,size
0,DM,F,1,236
1,DM,F,2,247
2,DM,F,3,47
3,DM,F,4,460
4,DM,F,5,212
...,...,...,...,...
135,PP,M,20,60
136,PP,M,21,43
137,PP,M,22,24
138,PP,M,23,3


In [81]:
wide = summary.pivot_table(index='plot_id', columns=['species_id', 'sex'], values='size', fill_value='')
wide.head()

species_id     DM            DO            PP       
sex             F      M      F      M      F      M
plot_id                                             
1           236.0  367.0  195.0  291.0  137.0   81.0
2           247.0  329.0  158.0  155.0   82.0  101.0
3            47.0   56.0   44.0   38.0  205.0  155.0
4           460.0  607.0   22.0   40.0   58.0   55.0
5           212.0  228.0   43.0   55.0   25.0   34.0

## Combining DataFrames

In general, there are four functions/methods for combining DataFrames, each of which has its own typical use (though the possibilities overlap):

- `concat` is a universal function for concatenating two or more tables/columns - underneath, next to each other, with or without indexes.
- `append` (method) is a simpler alternative to concat if you just want to add a few rows to a table.
- `merge` is a general-purpose function for joining tables based on index or column relationships.
- `join` (method) simplifies the job when you want to join two tables based on an index.

In [82]:
a = pd.Series(["one", "two", "three"])
b = pd.Series(["four", "five", "six"])

In [83]:
a.append(b)

0      one
1      two
2    three
0     four
1     five
2      six
dtype: object

Note that index repeating. We've created two Series where we haven't addressed the index. But pandas did, so it merged the two indexes, even at the cost of duplicate values. This can be avoided at by using the additional argument `ignore_index=True`, which we'll show with an example of joining two tables with the same columns:

In [84]:
df1 = pd.DataFrame({
    "left": [1, 2],
    "right": [3, 4]
})
df2 = pd.DataFrame({
    "left": [25, 26],
    "right": [47, 48]
})
df1.append(df2, ignore_index=True)

,left,right
0,1,3
1,2,4
2,25,47
3,26,48


The same using `pd.concat()`:

In [85]:
pd.concat([a, b])

0      one
1      two
2    three
0     four
1     five
2      six
dtype: object

In [86]:
pd.concat([a, a, a], ignore_index=True)

0      one
1      two
2    three
3      one
4      two
5    three
6      one
7      two
8    three
dtype: object

In [87]:
pd.concat([a, a, b, b], axis="columns")  # side by side

,0,1,2,3
0,one,one,four,four
1,two,two,five,five
2,three,three,six,six


### Joining dataframes
When we concatenated our DataFrames we simply added them to each other - stacking them either vertically or side by side. Another way to combine DataFrames is to use columns in each dataset that contain common values (a common unique id). Combining DataFrames using a common field is called “joining”. The columns containing the common values are called “join key(s)”. Joining DataFrames in this way is often useful when one DataFrame is a “lookup table” containing additional data that we want to include in the other.

For example, the species.csv file that is a lookup table. This table contains the genus, species and taxa code for 55 species. The species code is unique for each line. These species are identified in the survey data as well using the unique species code. Rather than adding 3 more columns for the genus, species and taxa to each of the 35,549 line Survey data table, we can maintain the shorter table with the species information. When we want to access that information, we can create a query that joins the additional columns of information to the survey data.

In [88]:
species = pd.read_csv("data/species.csv")
species.head()

,species_id,genus,species,taxa
0,AB,Amphispiza,bilineata,Bird
1,AH,Ammospermophilus,harrisi,Rodent
2,AS,Ammodramus,savannarum,Bird
3,BA,Baiomys,taylori,Rodent
4,CB,Campylorhynchus,brunneicapillus,Bird


#### Inner join

In [89]:
merged_inner = pd.merge(left=surveys_df, 
                        right=species, 
                        left_on="species_id", 
                        right_on="species_id")  # inner join (default)
merged_inner

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa
0,1,7,16,1977,2,NL,M,32.0,NaN,Neotoma,albigula,Rodent
1,2,7,16,1977,3,NL,M,33.0,NaN,Neotoma,albigula,Rodent
2,22,7,17,1977,15,NL,F,31.0,NaN,Neotoma,albigula,Rodent
3,38,7,17,1977,17,NL,M,33.0,NaN,Neotoma,albigula,Rodent
4,72,8,19,1977,2,NL,M,31.0,NaN,Neotoma,albigula,Rodent
...,...,...,...,...,...,...,...,...,...,...,...,...
34781,28988,12,23,1998,6,CT,NaN,NaN,NaN,Cnemidophorus,tigris,Reptile
34782,35512,12,31,2002,11,US,NaN,NaN,NaN,Sparrow,sp.,Bird
34783,35513,12,31,2002,11,US,NaN,NaN,NaN,Sparrow,sp.,Bird
34784,35528,12,31,2002,13,US,NaN,NaN,NaN,Sparrow,sp.,Bird


In [90]:
surveys_df.shape

(35549, 9)

The result of an inner join of `surveys_df` and `species` is a new DataFrame that contains the combined set of columns from `surveys_df` and `species`. It only contains rows that have two-letter species codes that are the same in both the `surveys_df` and `species` DataFrames. 
The result merged_inner DataFrame contains all of the columns from `surveys_df` (record id, month, day, etc.) as well as all the columns from `species` (species_id, genus, species, and taxa).

Notice that `merged_inner` has fewer rows than `surveys_df`. This is an indication that there were rows in `surveys_df` with value(s) for `species_id` that do not exist as value(s) for species_id in `species`.

#### Left join

In [91]:
merged_left = pd.merge(left=surveys_df, 
                       right=species, 
                       how="left", 
                       left_on="species_id", 
                       right_on="species_id")
merged_left

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa
0,1,7,16,1977,2,NL,M,32.0,NaN,Neotoma,albigula,Rodent
1,2,7,16,1977,3,NL,M,33.0,NaN,Neotoma,albigula,Rodent
2,3,7,16,1977,2,DM,F,37.0,NaN,Dipodomys,merriami,Rodent
3,4,7,16,1977,7,DM,M,36.0,NaN,Dipodomys,merriami,Rodent
4,5,7,16,1977,3,DM,M,35.0,NaN,Dipodomys,merriami,Rodent
...,...,...,...,...,...,...,...,...,...,...,...,...
35544,35545,12,31,2002,15,AH,NaN,NaN,NaN,Ammospermophilus,harrisi,Rodent
35545,35546,12,31,2002,15,AH,NaN,NaN,NaN,Ammospermophilus,harrisi,Rodent
35546,35547,12,31,2002,10,RM,F,15.0,14.0,Reithrodontomys,megalotis,Rodent
35547,35548,12,31,2002,7,DO,M,36.0,51.0,Dipodomys,ordii,Rodent


#### Your turn:

Find which values of species_id do not occur in species DataFrame

#### Other join types
The pandas merge function supports two other join types:

- Right (outer) join: Invoked by passing `how='right'` as an argument. Similar to a left join, except all rows from the right DataFrame are kept, while rows from the left DataFrame without matching join key(s) values are discarded.
- Full (outer) join: Invoked by passing `how='outer'` as an argument. This join type returns the all pairwise combinations of rows from both DataFrames; i.e., the result DataFrame will NaN where data is missing in one of the dataframes. This join type is very rarely used.